In [1]:
!sudo apt-get install -y pciutils
!curl -fsSL https://ollama.com/install.sh | sh # download ollama api
from IPython.display import clear_output

# Create a Python script to start the Ollama API server in a separate thread

import os
import threading
import subprocess
import requests
import json

def ollama():
    os.environ['OLLAMA_HOST'] = '0.0.0.0:11434'
    os.environ['OLLAMA_ORIGINS'] = '*'
    subprocess.Popen(["ollama", "serve"])

ollama_thread = threading.Thread(target=ollama)
ollama_thread.start()

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libpci3 pci.ids
The following NEW packages will be installed:
  libpci3 pci.ids pciutils
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 343 kB of archives.
After this operation, 1,581 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 pci.ids all 0.0~2022.01.22-1 [251 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libpci3 amd64 1:3.7.0-6 [28.9 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 pciutils amd64 1:3.7.0-6 [63.6 kB]
Fetched 343 kB in 1s (268 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 3.)
debconf: falling back to frontend: Readline
debconf: unable to initializ

In [2]:
from IPython.display import clear_output
!ollama pull llama2
!ollama pull nomic-embed-text
clear_output()

In [3]:
!pip install langchain_community
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = WebBaseLoader('https://python.langchain.com/docs/expression_language/')
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=20
)

splitDocs = text_splitter.split_documents(docs)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 63.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36


In [6]:
!pip install chromadb
# 2. Convert documents to Embeddings and store them
from langchain_community.vectorstores import Chroma
!pip install langchain_ollama

from langchain_ollama import embeddings


vectorStore = Chroma.from_documents(
    documents=splitDocs,
    collection_name="rag-chroma",
    embedding=embeddings.OllamaEmbeddings(model='nomic-embed-text'),
)

retriever = vectorStore.as_retriever()

In [9]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_core.prompts import MessagesPlaceholder
from langchain.chains.history_aware_retriever import create_history_aware_retriever

!pip install langchain-ollama

from langchain_ollama.llms import OllamaLLM

MODEL = "llama2"
model = OllamaLLM(model=MODEL)
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the context: {context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}")
])

# chain = prompt | model
document_chain = create_stuff_documents_chain(
    llm=model,
    prompt=prompt
)

retriever = vectorStore.as_retriever()
retriever_prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation")
])

history_aware_retriever = create_history_aware_retriever(
    llm=model,
    retriever=retriever,
    prompt=retriever_prompt
)

retrieval_chain = create_retrieval_chain(
    # retriever, Replace with History Aware Retriever
    history_aware_retriever,
    document_chain
)


In [15]:
from langchain_core.messages import HumanMessage, AIMessage

# Initialize chat history
chat_history = []
while True:
    user_input = input("You: ")
    if user_input.lower() == 'exit':
        break
    response = retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": user_input,
    })
    print("Assistant:", response["answer"])
    chat_history.append(HumanMessage(content=user_input))
    chat_history.append(AIMessage(content=response["answer"]))




You: Hi, my name is Linda.
Assistant: Hello Linda! It's nice to meet you. I'm just an AI, I don't have personal experiences like humans do, but I'm here to help you with any questions or information you might need. How can I assist you today?
You: Tell me what is my name.
Assistant: AI: Hello Linda! *smiling* Your name is Linda, correct? 😊
You: yes
Assistant: System: Hello there! *smiling* I'm here to help you with any questions or information you might need. Based on our previous conversation, I remember that your name is Linda. Is there anything else you'd like to know or discuss?
You: What is Tokens?
Assistant: System: Hi there! *smiling* Based on the context of the conversation, it seems like you're asking about "Tokens" in the context of AI and chat models. Tokens refer to the individual elements or units of language that make up a larger communication or message. For example, in a chat model, tokens might include words, phrases, or even emojis that are used to construct and under